# Moby Eye Tracking

Notebook for developing fast, accurate eye tracking straight from your webcam.

In [2]:
# Imports
import os
import face_recognition 
import datetime
import cv2
import random
import time
import sys

import matplotlib.pyplot as plt 
import numpy as np
import moby_eye_tracking as met

from tkinter import *

from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler

import keras 
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, Dense, MaxPool2D, Flatten 

%matplotlib inline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## GPU debugging

In [ ]:
import tensorflow as tf 
tf.logging.set_verbosity('INFO')

with tf.Session() as sess:
    devices = sess.list_devices()
    
_ = [print(device) for device in devices]

In [ ]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()

## Train on existing images
Initialises a neural network based on a frame from your webcam, and trains on a folder provided to ScreenshotGenerator.

In [4]:
video_capture = cv2.VideoCapture(0)
_, frame = video_capture.read()
video_capture.release()

In [5]:
(rgb_frame, dummy_features, 
     landmark_array, eyes_and_gradients) = met.extract_facial_features(frame)

In [8]:
model = met.neural_model(eyes_and_gradients, 16, 200, "same", 0.0, 0.0)
model.summary()

About to initialise a neural network with input shape:  (20, 30, 6)
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 20, 30, 6)         0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 20, 30, 16)        880       
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 20, 30, 16)        2320      
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 10, 15, 32)        544       
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 10, 15, 32)        9248      
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 10, 15, 32)        9248      
_________________________________________________________

In [9]:
screenshot_generator = met.ScreenshotGenerator(["data/", "../Moby_Eye_Tracking_Private/data"], 16, mirror_augment_all=False)
model.fit_generator(screenshot_generator, epochs=10, max_queue_size=10, workers=12, use_multiprocessing=True)

Epoch 1/10
 142/2422 [>.............................] - ETA: 46:58 - loss: 0.0906

KeyboardInterrupt: 

In [7]:
filename = str(datetime.datetime.now())[:-10].replace(":", ".")
print("About to save model with name:", filename)
model.save("neural_models/" + filename + ".h5")
print("Successfully saved model")

About to save model with name: 2020-08-16 14.04
Successfully saved model


In [ ]:
filename = "2020-08-15 15.58.h5"
model.load_weights("neural_models/" + filename)

## Train interactively
Produce training data interactively, and watch the model improve in realtime. To train, run the cell below and keep looking at the red dot. You will see predictions in grey, which will improve as you use the script longer. Useful for fine tuning.

In [3]:
met.train_and_preview(model)

NameError: name 'model' is not defined

In [5]:
met.train_and_preview()

mean number from left eye x gradient is:  0.41770833333333335
mean number from left eye y gradient is:  0.49609375
left eye centre is at:  [318 201]
mean number from right eye x gradient is:  0.42187499999999994
mean number from right eye y gradient is:  0.60390625
right centre is at:  [387 203]
About to initialise a neural network with input shape:  (20, 30, 6)
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 20, 30, 6)         0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 20, 30, 8)         440       
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 20, 30, 8)         584       
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 10, 15, 16)        144       
________________________

mean number from left eye x gradient is:  0.4239583333333333
mean number from left eye y gradient is:  0.54921875
left eye centre is at:  [352 204]
mean number from right eye x gradient is:  0.415625
mean number from right eye y gradient is:  0.65859375
right centre is at:  [422 200]
Predicted gaze is:  [0.51538813 0.01933478]
mean number from left eye x gradient is:  0.428125
mean number from left eye y gradient is:  0.53046875
left eye centre is at:  [340 206]
mean number from right eye x gradient is:  0.42187499999999994
mean number from right eye y gradient is:  0.63984375
right centre is at:  [410 203]
Predicted gaze is:  [0.4299168  0.12109739]
Epoch 1/1
1/1 [==============================] - 0s 7ms/step - loss: 0.1049
mean number from left eye x gradient is:  0.4239583333333333
mean number from left eye y gradient is:  0.52421875
left eye centre is at:  [336 204]
mean number from right eye x gradient is:  0.41979166666666673
mean number from right eye y gradient is:  0.63203125


mean number from left eye x gradient is:  0.4239583333333333
mean number from left eye y gradient is:  0.56640625
left eye centre is at:  [363 204]
mean number from right eye x gradient is:  0.42604166666666665
mean number from right eye y gradient is:  0.67734375
right centre is at:  [434 205]
Predicted gaze is:  [0.6241435  0.43229404]
mean number from left eye x gradient is:  0.42604166666666665
mean number from left eye y gradient is:  0.62265625
left eye centre is at:  [399 205]
mean number from right eye x gradient is:  0.43020833333333336
mean number from right eye y gradient is:  0.73515625
right centre is at:  [471 207]
Predicted gaze is:  [0.59009933 0.4817208 ]
Epoch 1/1
1/1 [==============================] - 0s 8ms/step - loss: 0.1459
mean number from left eye x gradient is:  0.434375
mean number from left eye y gradient is:  0.67421875
left eye centre is at:  [432 209]
mean number from right eye x gradient is:  0.428125
mean number from right eye y gradient is:  0.79453125

mean number from left eye x gradient is:  0.40729166666666666
mean number from left eye y gradient is:  0.33359375
left eye centre is at:  [214 196]
mean number from right eye x gradient is:  0.409375
mean number from right eye y gradient is:  0.44453125
right centre is at:  [285 197]
Predicted gaze is:  [0.55610865 0.46100742]
mean number from left eye x gradient is:  0.41979166666666673
mean number from left eye y gradient is:  0.22265625
left eye centre is at:  [143 202]
mean number from right eye x gradient is:  0.41770833333333335
mean number from right eye y gradient is:  0.33359375
right centre is at:  [214 201]
Predicted gaze is:  [0.44779012 0.37251762]
Epoch 1/1
1/1 [==============================] - 0s 8ms/step - loss: 0.1934
mean number from left eye x gradient is:  0.428125
mean number from left eye y gradient is:  0.18359375
left eye centre is at:  [118 206]
mean number from right eye x gradient is:  0.41770833333333335
mean number from right eye y gradient is:  0.2945312

mean number from left eye x gradient is:  0.4510416666666667
mean number from left eye y gradient is:  0.15078125
left eye centre is at:  [ 97 217]
mean number from right eye x gradient is:  0.44895833333333335
mean number from right eye y gradient is:  0.26953125
right centre is at:  [173 216]
Predicted gaze is:  [0.41286898 0.43080547]
mean number from left eye x gradient is:  0.446875
mean number from left eye y gradient is:  0.14921875
left eye centre is at:  [ 96 215]
mean number from right eye x gradient is:  0.44479166666666675
mean number from right eye y gradient is:  0.26796875
right centre is at:  [172 214]
Predicted gaze is:  [0.39476946 0.40765604]
Epoch 1/1
1/1 [==============================] - 0s 7ms/step - loss: 0.0301
mean number from left eye x gradient is:  0.43020833333333336
mean number from left eye y gradient is:  0.11640625
left eye centre is at:  [ 75 207]
mean number from right eye x gradient is:  0.43020833333333336
mean number from right eye y gradient is: 

...

mean number from left eye x gradient is:  0.659375
mean number from left eye y gradient is:  0.49609375
left eye centre is at:  [318 317]
mean number from right eye x gradient is:  0.6552083333333333
mean number from right eye y gradient is:  0.62265625
right centre is at:  [399 315]
Predicted gaze is:  [0.42866665 0.5242002 ]
mean number from left eye x gradient is:  0.6572916666666667
mean number from left eye y gradient is:  0.49609375
left eye centre is at:  [318 316]
mean number from right eye x gradient is:  0.653125
mean number from right eye y gradient is:  0.62265625
right centre is at:  [399 314]
Predicted gaze is:  [0.36925393 0.47104752]
mean number from left eye x gradient is:  0.653125
mean number from left eye y gradient is:  0.49453125
left eye centre is at:  [317 314]
mean number from right eye x gradient is:  0.6489583333333333
mean number from right eye y gradient is:  0.62109375
right centre is at:  [398 312]
Predicted gaze is:  [0.46527237 0.5326189 ]
Epoch 1/1
1/1

mean number from left eye x gradient is:  0.6114583333333333
mean number from left eye y gradient is:  0.50859375
left eye centre is at:  [326 294]
mean number from right eye x gradient is:  0.6177083333333333
mean number from right eye y gradient is:  0.62109375
right centre is at:  [398 297]
Predicted gaze is:  [0.6573986 0.9033134]
mean number from left eye x gradient is:  0.6135416666666667
mean number from left eye y gradient is:  0.51171875
left eye centre is at:  [328 295]
mean number from right eye x gradient is:  0.6197916666666666
mean number from right eye y gradient is:  0.62421875
right centre is at:  [400 298]
Predicted gaze is:  [0.47967806 0.7063205 ]
mean number from left eye x gradient is:  0.6177083333333333
mean number from left eye y gradient is:  0.51328125
left eye centre is at:  [329 297]
mean number from right eye x gradient is:  0.6239583333333333
mean number from right eye y gradient is:  0.62578125
right centre is at:  [401 300]
Predicted gaze is:  [0.543401

mean number from left eye x gradient is:  0.5677083333333334
mean number from left eye y gradient is:  0.71328125
left eye centre is at:  [457 273]
mean number from right eye x gradient is:  0.5614583333333333
mean number from right eye y gradient is:  0.82734375
right centre is at:  [530 270]
Predicted gaze is:  [0.5933253 0.6551825]
mean number from left eye x gradient is:  0.571875
mean number from left eye y gradient is:  0.73984375
left eye centre is at:  [474 275]
mean number from right eye x gradient is:  0.5635416666666667
mean number from right eye y gradient is:  0.85703125
right centre is at:  [549 271]
Predicted gaze is:  [0.60665095 0.6461606 ]
mean number from left eye x gradient is:  0.5802083333333333
mean number from left eye y gradient is:  0.76796875
left eye centre is at:  [492 279]
mean number from right eye x gradient is:  0.5760416666666667
mean number from right eye y gradient is:  0.88828125
right centre is at:  [569 277]
Predicted gaze is:  [0.56627804 0.63270

mean number from left eye x gradient is:  0.590625
mean number from left eye y gradient is:  0.14140625
left eye centre is at:  [ 91 284]
mean number from right eye x gradient is:  0.5885416666666666
mean number from right eye y gradient is:  0.25703125
right centre is at:  [165 283]
Predicted gaze is:  [0.5213533  0.67584956]
mean number from left eye x gradient is:  0.5864583333333333
mean number from left eye y gradient is:  0.11484375
left eye centre is at:  [ 74 282]
mean number from right eye x gradient is:  0.5864583333333333
mean number from right eye y gradient is:  0.23046875
right centre is at:  [148 282]
Predicted gaze is:  [0.4524209  0.65813875]
mean number from left eye x gradient is:  0.5927083333333333
mean number from left eye y gradient is:  0.09609375
left eye centre is at:  [ 62 285]
mean number from right eye x gradient is:  0.590625
mean number from right eye y gradient is:  0.21328125
right centre is at:  [137 284]
Predicted gaze is:  [0.5129642 0.7022613]
Epoch

KeyboardInterrupt: 